<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/05_data_compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: 04/04/2022

Date modified: April 16, 2024

Purpose: This compiles climate, land use, and flow diversion outputs and puts them in a usable, long-format csv file to import into R.


In [1]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
## -------------------- ##
## IMPORT ALL DATASETS ##
## -------------------- ##

div_files = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/diversion_timeseries_out/model_inputs.csv')
land_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/subset_LULCC_out/final_metrics/*.csv'))
climate_files = sorted(glob.glob('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/extract_gridmet_out/final/*.csv'))
hydromet = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/hydromet_data_out/model_input_hydromet.csv')
common_name_flow = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/diversion_timeseries/relates/name_dictionary_flow.csv')
common_name_spatial = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/diversion_timeseries/relates/name_dictionary_spatial.csv')
POUSize = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/diversion_timeseries/relates/POUSize.csv')
storage = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/diversion_timeseries/WRA_BoiseBasin.csv')
quantiles = pd.read_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/diversion_timeseries/quantiles.csv')
ubrb_prcp = pd.read_csv('/content/gdrive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
water_rights = pd.read_csv('/content/gdrive/MyDrive/Data/Model Modifications/div_water_rights/final_div_wr.csv')
#irrigation_change = pd.read_csv('/content/gdrive/MyDrive/Data/Model Modifications/div_irrigation_change/div_irrigation_model_input.csv')

In [3]:
## --------------------------------- ##
## CREATE COMMON NAME TO MERGE FILES ##
## --------------------------------- ##

div = div_files

div_dict = dict(zip(common_name_flow['DiversionName'], common_name_flow['NewName']))
div['NewName'] = div['Name'].map(div_dict)
div = div.drop(['Unnamed: 0', 'Name'], axis=1).rename({'NewName' : 'Name'}, axis = 1)


# Import land use change data

land_data = []
for i in land_files:
  land_data.append(pd.read_csv(i))

land = pd.concat(land_data)

# Land use dictionary

shape_dict = dict(zip(common_name_spatial['WaterRight'], common_name_spatial['NewName']))
land['Name'] = land['DivName'].map(shape_dict)
land = land.drop(['Unnamed: 0', 'DivName'], axis=1)
POUSize['Name'] = POUSize['WaterRight'].map(shape_dict)
POUSize = POUSize.drop(['WaterRight'], axis=1)

# Import climate zonal stats

clim_data = []
for i in climate_files:
  clim_data.append(pd.read_csv(i))
clim = pd.concat(clim_data)

# Use shapefile dictionary on climate data

clim['Name'] = clim['DIV_NAME'].map(shape_dict)
clim = clim.drop(['Unnamed: 0', 'DIV_NAME'], axis=1)

# UBRB precip
ubrb_prcp = ubrb_prcp.rename(columns={'year': 'Year'}) #rename to fit column structure
ubrb_prcp = ubrb_prcp.rename(columns = {'precip':'ubrb_prcp'}) #rename to fit column structure

# Irrigation Change
#irrigation_change = irrigation_change.rename(columns = {'Predicted_Proportion': 'pivot_prop'}) #rename to fit column structure

# Water Rights
water_rights['Year'] = pd.to_numeric(water_rights['Year'], errors='coerce')
water_rights = water_rights.rename(columns = {'Diversion': 'Name'}) #rename to fit column structure

In [4]:
## -------------------------------------- ##
## MERGE DIVERSION DATA TO MATCH THE POUS ##
## -------------------------------------- ##

# Create function to do this

def merge_flows(data, name):
  '''
  This function will merge two different flow datasets into one for completely overlapping POUs.

  Variables:
  data : The full diversion dataset
  name : A string of the new name for each POU.
  '''
  old_df = data[data['Name']== name].reset_index().drop('index', axis=1)
  new_df = pd.DataFrame()
  new_df['Year'] = old_df['Year'].unique()
  new_df['Name'] = old_df['Name'][0:34]
  sums = old_df.groupby('Year').sum().reset_index()
  new_df['Diversion (cfs)'] = sums['Diversion (cfs)']
  new_df['Acre_feet'] = sums['Acre_feet']
  # quants = old_df['Quantiles'].groupby('Year').max().reset_index()

  startday = []
  start_date = []
  endday = []
  range = []
  end_date = []
  censored = []

  for i in new_df['Year']:
    yearly = old_df[old_df['Year'] == i]
    start = np.min(yearly['StartDayofYear'].values)
    startdate = yearly['StartDate'][yearly['StartDayofYear']==start].values
    end = np.max(yearly['EndDayofYear'].values)
    enddate = yearly['EndDate'][yearly['EndDayofYear']==end].values
    censor = str(yearly['Censor'].head(1))
    startday.append(start)
    endday.append(end)
    range.append(end-start)
    start_date.append(startdate[0])
    end_date.append(enddate[0])
    censored.append(censor)

  new_df['StartDate'] = start_date
  new_df['StartDayofYear'] = startday
  new_df['EndDate'] = end_date
  new_df['EndDayofYear'] = endday
  new_df['Time'] = range
  new_df['Censor'] = censored

  return new_df


In [5]:
## ------------------------------------- ##
## MERGE DIVERSION DATASET WITH NEW DATA ##
## ------------------------------------- ##

# Create a list of names that have completely shared POUs
merge_names = ['Shipley and Wagner Pumps', 'Rossi Mill and Meeves Canals', 'Boise City Parks']

merged = []
for i in merge_names:
  new = merge_flows(div, i)
  div = div[div['Name'] != i] #Remove old dataframes from full dataset
  div = pd.concat([div, new])
  merged.append(new)

div = div.sort_values(by=['Name', 'Year']).reset_index().drop('index',axis=1)


In [6]:
## ---------------------------------- ##
##  MERGE ALL FILES INTO ONE FILE     ##
## ---------------------------------- ##

land_div = div.merge(land, left_on=['Year', 'Name'], right_on=['dates','Name'], how='left')
full_df = land_div.merge(clim, left_on=['Year','Name'], right_on=['Year', 'Name'], how='left').sort_values(by=['Name', 'Year'])
full_df = full_df.merge(hydromet, left_on='Year', right_on='Year', how='left').drop(['Unnamed: 0', 'dates'], axis=1)
full_df = full_df.merge(POUSize, left_on = 'Name', right_on = 'Name', how = 'left')
full_df = full_df.merge(storage, left_on = ['Year', 'Name'], right_on = ['Year', 'Name'])


full_df = full_df.merge(water_rights, on = ['Year', 'Name'], how = 'left')
#full_df = full_df.merge(irrigation_change, on = ['Year', 'Name'], how = 'left')
full_df.fillna(0, inplace=True) #fills empty cells with 0
full_df = full_df.merge(ubrb_prcp, on='Year', how='inner')
#full_df = quantiles.merge(full_df, left_on = ['Year', 'Name'], right_on = ['Year', 'Name'])
merge_final = pd.merge(full_df, hydromet, on='Year', how='left') #adds in water year ubrb prcp for each diversion

In [7]:
## ------------------------------- ##
##  Remove/Rename some columns     ##
## ------------------------------- ##

merge_final.columns
merge_final = merge_final.drop(['Unnamed: 0','Unnamed: 5', 'Unnamed: 6', 'LP_inflows_y','Max_Fill_y', 'Carryover_y'], axis=1)
merge_final = merge_final.rename({'LP_inflows_x': 'LP_inflows', 'Max_Fill_x': 'Max_Fill', 'Carryover_x': 'Carryover'}, axis=1)

In [8]:
## --------------------------------------- ##
## Export the full csv file for model in R ##
## --------------------------------------- ##

# Specify the order of columns for later analysis scripts
desired_order = ['Year', 'Acre_feet', 'Diversion (cfs)', 'Irrigation Year', 'SiteID', 'Month', 'DayofYear','Sum','StartDate','StartDayofYear','EndDate','EndDayofYear','Time','Censor','_merge','Name','class1_urban','class2_crops','contagion','largest_patch_index','ant_prcp','irrig_prcp','irrig_temp','JuneAug_temp','et','Mar_et','Mar_tmp','Mar_prcp','LP_inflows','Max_Fill','Carryover','Size_acres','AF_used','AF_remaining','AF_available', 'ubrb_prcp', 'gw_wr','sw_wr', 'total_wr']
merge_final = merge_final[desired_order]

out_path = '/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/data_compilation/merged/model_input_0906.csv'
merge_final.to_csv(out_path)

# Individual dataframe export

individual_diversions = merge_final['Name'].unique()
for i in individual_diversions:
  df = merge_final[merge_final['Name'] == i]
  out_path = os.path.join('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_output/'+i+'.csv')
  df.to_csv(out_path)